# Решение кейса "Рекомендательная система видео" 
## Кейсодержатель: RUTUBE

In [180]:
import pandas as pd
import os
import gc

print("loading data ...")
hack = pd.read_csv("small_player_starts_train.csv")

loading data ...


In [181]:
hack.head(5)

,date,user_id,item_id,watch_time,is_autorized
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0
1,2023-08-21 16:17:58+03:00,user_17893165,video_796847,374,0
2,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0
3,2023-08-21 01:06:10+03:00,user_24477183,video_1156618,1,0
4,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0


#### Для решения возьмем данные о просмотрах пользователей и метаданные о видео

In [182]:
print("Amount of events: ", len(hack))
users = hack.user_id.unique()
print("Unique users: ", len(users))
print("Unique items: ", hack.item_id.nunique())
print("Min date: ", hack.date.min())
print("Max date: ", hack.date.max())

Amount of events:  2439644
Unique users:  1313891
Unique items:  261288
Min date:  2023-08-21 00:00:00+03:00
Max date:  2023-08-21 23:59:59+03:00


In [183]:
# preprocess date
hack['date_short'] = hack.date.apply(lambda l: l.split(" ")[0])

#### Отфильтруем только "активных" пользователей (тех, у кого более 1 просмотра за сессию, в нашем случае - за сутки).

In [184]:
%%time
groupby = hack.groupby(["user_id", "date_short"]).count()
users_active = groupby[groupby['item_id'] >= 2].index.unique()

CPU times: user 2.54 s, sys: 134 ms, total: 2.67 s
Wall time: 2.76 s


In [185]:
users_active = set(users_active)

In [186]:
from tqdm import tqdm 
tqdm.pandas()
hack['active'] = hack.progress_apply(lambda l: True if (l.user_id, l.date_short) in users_active else False, axis=1)
hack=hack[hack.active]

100%|██████████████████████████████| 2439644/2439644 [00:25<00:00, 95408.01it/s]


## Необходимые датасеты

In [ ]:
hack.reset_index(drop= True , inplace= True )

In [188]:
hack

,date,user_id,item_id,watch_time,is_autorized,date_short,active
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0,2023-08-21,True
1,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0,2023-08-21,True
2,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0,2023-08-21,True
3,2023-08-21 15:21:19+03:00,user_22148320,video_1366097,32,0,2023-08-21,True
4,2023-08-21 00:35:41+03:00,user_14452025,video_283933,2142,0,2023-08-21,True
...,...,...,...,...,...,...,...
1457171,2023-08-21 16:44:47+03:00,user_11895299,video_853811,101,0,2023-08-21,True
1457172,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1,2023-08-21,True
1457173,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0,2023-08-21,True
1457174,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0,2023-08-21,True


In [9]:
features = pd.read_parquet("videos.parquet")

In [179]:
features

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С М...,Silvi,None,0,В видео я обывательским взглядом расскажу про ...,Технологии и интернет,2022-12-08 13:53:05+03:00,391382,0,0,NaN,0.0,0.000000,NaN
1,video_1173704,Наложение пястно фаланговой повязки на кисть,"УЦ ""Академия Безопасности""",None,0,Видео с канала УЦ Академия безопасности (ab-dp...,Образование,2022-03-24 09:19:15+03:00,125922,26,0,NaN,0.0,0.000000,0.00
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Artur48,None,0,SilverstoneF1 Sochi Pro и Neoline x cop 6000s ...,Авто-мото,2022-03-19 17:41:49+03:00,436570,2,0,NaN,NaN,0.000000,0.00
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИ...,ПАПА ДОЧКИ Games,None,0,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookha...,Детям,2021-02-20 11:50:53+03:00,719377,673,0,0.0,1.0,0.000000,0.00
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Б...","ЗАО ""Сплайн-Центр""",None,0,"10.08.2023 Вебинар ""Особенности трудоустройств...",Бизнес и предпринимательство,2023-08-11 09:02:07+03:00,3834404,19,0,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320655,video_839160,Любимцы: Гена в памперсе бегает по клинике,Любимцы,Любимцы,1,None,Сериалы,2017-09-11 22:00:13+03:00,23290,0,0,NaN,0.0,0.333333,0.25
2320656,video_2134427,"Телеканал «СПАС», «Главное», интервью С. В. Ст...",Фонд развития территорий,None,0,None,Интервью,2022-03-28 11:19:33+03:00,1548792,57,0,0.0,0.0,0.000000,0.00
2320657,video_1453482,Взятие 3 отметок на ИС 7,CeperA_TyLyH,None,0,None,Видеоигры,2022-09-10 20:05:17+03:00,520001,12,0,NaN,NaN,NaN,NaN
2320658,video_2225207,Простой и вкусный рецепт пирога с грушами и сы...,Комедийные Шоу и Скетчи ❤,None,0,None,Разное,2023-07-06 12:16:29+03:00,8267,9,0,0.0,0.0,0.000000,0.00


### Подготовка данных

In [382]:
import numpy as np

item = np.array(features['item_id'].to_list())
item_h = np.array(hack['item_id'].to_list())
categ = np.array(features['category_title'].to_list())
authors = np.array(features['author_title'].to_list())
hack1 = hack
hack1

In [355]:
%%time
hack1['cat'] = [categ[np.where(item == i)][0] for i in tqdm(item_h)]

100%|███████████████████████████████████████| 1000/1000 [00:12<00:00, 78.52it/s]

CPU times: user 12.7 s, sys: 53.1 ms, total: 12.8 s
Wall time: 12.7 s



<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
hack1['author'] = [authors[np.where(item == i)][0] for i in tqdm(item_h)]

 25%|███████▌                      | 368941/1457176 [1:19:59<3:55:58, 76.86it/s]

## Модель
### Построим модель рекомендаций, основанную на совместной встречаемости видео, категорий видео и определенных авторов в сессиях пользователей

В качеcтве сессии будем брать просмотры пользователя за сутки

In [358]:
def calc_pairs_video(train):
    print("Get sessions")
    count = train.groupby(['item_id']).size().rename('count_source').reset_index()[
        ['item_id', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['item_id'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'item_id']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['item_id', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['item_id'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['item_id', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['item_id'], how='left')
    count = count.rename({'count_source': 'count_target', 'item_id': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [359]:
def calc_pairs_category(train):
    print("Get sessions")
    count = train.groupby(['cat']).size().rename('count_source').reset_index()[
        ['cat', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['cat'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'cat']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['cat', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['cat'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['cat', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['cat'], how='left')
    count = count.rename({'count_source': 'count_target', 'cat': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [360]:
def calc_pairs_authors(train):
    print("Get sessions")
    count = train.groupby(['author']).size().rename('count_source').reset_index()[
        ['author', 'count_source']]
    dt = train.groupby(['user_id', 'date_short'])['author'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'author']].merge(dt, on=['user_id', 'date_short'], how='left')
        
    
    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['author', 'pair']].explode(column='pair')
    gc.collect()
    
    print("Discard duplicates")
    df = df.loc[df['author'] != df['pair']].reset_index(drop=True)
    
    print("Count how many times each pair combination happens")
    df = df.groupby(['author', 'pair']).size().rename('count').reset_index()
    
    print("join with frecuency of source item")
    df = df.merge(count, on=['author'], how='left')
    count = count.rename({'count_source': 'count_target', 'author': 'pair'}, axis=1)
    df = df.merge(count, on=['pair'], how='left')

    df['score'] = df.apply(lambda l: l['count'] / (l['count_source']*l['count_target']), axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [361]:
%%time
video = calc_pairs_video(hack1[['user_id', 'item_id', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 482 ms, sys: 271 ms, total: 753 ms
Wall time: 949 ms


In [ ]:
video

In [362]:
%%time
categories = calc_pairs_category(hack1[['user_id', 'cat', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 398 ms, sys: 5.95 ms, total: 404 ms
Wall time: 404 ms


In [ ]:
categories

In [363]:
%%time
authors = calc_pairs_authors(hack1[['user_id', 'author', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
join with frecuency of source item
CPU times: user 394 ms, sys: 19.9 ms, total: 414 ms
Wall time: 412 ms


In [366]:
authors

,cat,pair,count,count_source,count_target,score
0,Животные,Спорт,1,38,28,0.000940
1,Спорт,Животные,1,28,38,0.000940
2,Телепередачи,Юмор,1,369,43,0.000063
3,Юмор,Телепередачи,1,43,369,0.000063


# Submission

### Cделаем предсказания для тестовых пользователей, будем притягивать рекомендации к последнему видео в трейне следующим образом: к последнему видео рекомендуется ряды видео категорий и авторов, часто встречаемых с автором и категорией данного ролика

Для пользователей, которые не попали в трейн, но есть на тесте, порекомендуем самые популярные видео из трейна

#### Функции для поиска категорий и авторов 

In [369]:
def auth_pop(author, features, num: int=5):
    auth = features.loc[features['author_title'] == author]['item_id'].sort_values(by = ['publicated'], ascending = [False])
    return auth['item_id'].tolist()[:num]
    
    
def cat_pop(cat, features, num: int=5):
    cats = features.loc[features['category_title'] == cat]['item_id'].sort_values(by = ['publicated'], ascending = [False])
    return cats['item_id'].tolist()[:num]

#### Самые популярные видео для холодных пользователей

In [370]:
most_popular = list(hack1.item_id.value_counts()[:15].to_frame().index)

In [371]:
features[features.item_id.isin(most_popular)][:5]

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
155423,video_68646,"Выжить в Дубае, 8 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-08-13 21:00:12+03:00,5208840,75496,0,NaN,NaN,NaN,0.411112
238036,video_956001,Прямой эфир ТНТ4,ТНТ4,ТНТ4 live,0,"Смелый, яркий и очень смешной канал ТНТ4 можно...",Телепередачи,2019-06-07 14:40:38+03:00,0,798,0,0.054545,NaN,NaN,0.000000
711554,video_302657,"Выжить в Дубае, 6 выпуск",Телеканал ТНТ,Выжить в Дубае,1,Премьера! «Выжить в Дубае» – новое масштабное ...,Телепередачи,2023-07-30 21:00:06+03:00,5559360,75496,0,NaN,0.491602,0.379315,0.289821
755756,video_1733917,Как продлить стойкость макияжа в летний период,Juuuls,None,0,У всех у нас с приходом лета возникает один во...,Красота,2023-08-16 14:55:26+03:00,367267,17483,0,NaN,NaN,NaN,0.000000
881802,video_2095591,РИСУЕМ ОКЕАН И РЫБ НА СТЕНЕ! - реакция заказчи...,marialalart,None,0,Сегодня мы с вами расписываем стенку в апарта...,Искусство,2023-08-14 19:54:50+03:00,447703,3343,0,NaN,NaN,NaN,0.000141


#### Находим последнее просмотренное видео пользователями из теста (которые также есть и в трейне)

In [372]:
hack1 = hack1[hack1.user_id.isin(hack1['user_id'].unique())].sort_values(by=['date'])

In [373]:
%%time
last_watched_videos = (
    hack[['user_id', 'item_id']]
    .groupby(['user_id'], as_index=False)
    .agg('last')
)

CPU times: user 692 ms, sys: 349 ms, total: 1.04 s
Wall time: 2.38 s


In [374]:
last_watched_videos = last_watched_videos.set_index('user_id').to_dict()['item_id']

In [375]:
items_in_train = {source: list(group.index) for source, group in rules.groupby(['item_id'])}
cats_in_train = {source: list(group.index) for source, group in categories.groupby(['cat'])}
authors_in_train = {source: list(group.index) for source, group in authors.groupby(['author'])}

### Решение

In [376]:
sub = pd.read_csv('sample_submission.csv')

In [378]:
from tqdm import tqdm 
tqdm.pandas()


def get_recs_for_users(rules, authors, categories,  sample_submission, last_watched_videos, items_in_train, num_candidates: int=3):
    
    for i, row in tqdm(sample_submission.iterrows()):
        if row.user_id in last_watched_videos:
            video = last_watched_videos[row.user_id]
            video_ids = items_in_train[video] if video in items_in_train else ''
            aut_ids = authors_in_train[features.loc[features['item_id'] == video]['author_title'].values[0]] if features.loc[features['item_id'] == video]['author_title'].values[0] in authors_in_train else ''
            cat_ids = cats_in_train[features.loc[features['item_id'] == video]['category_title'].values[0]] if features.loc[features['item_id'] == video]['category_title'].values[0] in cats_in_train else ''
            if len(video_ids)>0:
                recs = []
                recs.extend(rules.iloc[video_ids].item_id.to_list()[:num_candidates])
                auth = authors.iloc[aut_ids].item_id.to_list()[:num_candidates]
                catts = categories.iloc[cat_ids].item_id.to_list()[:num_candidates]
                for i in range(num_candidates):
                    recs.extend([auth_pop(auth[i], features, num_candidates + 1), cat_pop(catts[i], features, num_candidates)])
                sample_submission.loc[i, 'recs'] = recs
            else:
                sample_submission.loc[i, 'recs'] = most_popular
        else:
            sample_submission.loc[i, 'recs'] = most_popular
    return sample_submission

In [380]:
submission = get_recs_for_users(rules, authors, categories, sub, last_watched_videos, items_in_train)

0it [00:00, ?it/s]/var/folders/kh/xsjpwqw577x7d3m333j9gb3h0000gn/T/ipykernel_41043/2135796989.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_submission.loc[i, 'recs'] = most_popular
1it [00:00,  3.84it/s]/var/folders/kh/xsjpwqw577x7d3m333j9gb3h0000gn/T/ipykernel_41043/2135796989.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_submission.loc[i, 'recs'] = most_popular
500it [00:33, 14.92it/s]


In [381]:
submission

,user_id,recs
0,user_26511551,"[video_283933, video_1508623, video_68646, vid..."
1,user_29194819,"[video_283933, video_1508623, video_68646, vid..."
2,user_29734049,"[video_283933, video_1508623, video_68646, vid..."
3,user_955460,"[video_283933, video_1508623, video_68646, vid..."
4,user_7065521,"[video_283933, video_1508623, video_68646, vid..."
...,...,...
495,user_19042269,"[video_283933, video_1508623, video_68646, vid..."
496,user_29494756,"[video_283933, video_1508623, video_68646, vid..."
497,user_20271628,"[video_283933, video_1508623, video_68646, vid..."
498,user_2600551,"[video_283933, video_1508623, video_68646, vid..."


In [35]:
submission.to_csv("submission.csv", index=False)